Übung: Blue/Green Deployments
-----------------

In dieser Übung führen wir einen Blue/Green Deployment durch.

Dazu wird ein weiteres Deployment mit der neuen Version vom BPMN Frontend erstellt und mittels Veränderung der Zusteuerung des Service zwischen den Version hin und her geschaltet.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace bluegreen

Deployment und Services wurde in einzelne Dateien aufgeteilt. Die Ingress Ressource, aus Einfachheitsgründen weggelassen.

Die Labels und das Image wurden um eine Version Angabe erweitert:

    apiVersion: v1
    kind: Service
    metadata:
      name: bpmn-frontend
      labels:
        app: bpmn-frontend
    spec:
      ...
      selector:
        app: bpmn-frontend
        version: "0.2"    
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: bpmn-frontend
      version: 0.2
    ...
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/misegr/bpmn-frontend:V0.2
        imagePullPolicy: IfNotPresent       



In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-green.yaml --namespace bluegreen
! kubectl apply -f 09-4-Deployment/bpmn-frontend-service-green.yaml --namespace bluegreen

Das Ergebnis unterscheidet sich, vorerst, nicht von einem normalen Deployment

In [ ]:
! kubectl get all --namespace bluegreen
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace bluegreen bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

- - -

Für das **blue** Deployment erstellen wir nur ein Deployment aber kein Service:

In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-blue.yaml --namespace bluegreen
! kubectl get all --namespace bluegreen -o wide

Der Service steuert auf die **green** Pods zu.

Erst nach Ändern des selectors auf die **blue** Version wird diese für den Anwender sichtbar

        apiVersion: v1
        kind: Service
        metadata:
          name: bpmn-frontend
          labels:
            app: bpmn-frontend
        spec:
          ...
          selector:
            app: bpmn-frontend
            version: "1.0"
        

In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-service-blue.yaml --namespace bluegreen

An den Anzahl Ressourcen ändert der Selector nichts

In [ ]:
! kubectl get all --namespace bluegreen -o wide

- - -
Nach den erfolgreichen Tests der neuen Version können wir das **green** Deployment löschen.

In [ ]:
! kubectl delete -f 09-4-Deployment/bpmn-frontend-deployment-green.yaml --namespace bluegreen
! kubectl get all --namespace bluegreen

- - -

Aufräumen

In [ ]:
! kubectl delete namespace bluegreen